# 1.DATA

In [ ]:
import numpy as np
from datetime import datetime as dt
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
with open ('sx-stackoverflow-a2q.txt') as dataset_a2q:
    a2q=dataset_a2q.readlines()

with open ('sx-stackoverflow-c2a.txt') as dataset_c2a:
    c2a= dataset_c2a.readlines()
    
with open ('sx-stackoverflow-c2q.txt') as dataset_c2q:
    c2q= dataset_c2q.readlines()

### Preparing the data

Note that we're changing the values of the read data instead of creating new variables to store the corrected version, in order to save space.

In [ ]:
n=100 #we'll try first with n=100 rows from each one of the 3 datasets
for i in range(0,n):
    a2q[i]=a2q[i].split()
    a2q[i][0]=int(a2q[i][0])
    a2q[i][1]=int(a2q[i][1])
    a2q[i][2]=dt.utcfromtimestamp(int(a2q[i][2])).strftime('%Y-%m-%d')
    
for i in range(0,n):
    c2a[i]=c2a[i].split()
    c2a[i][0]=int(c2a[i][0])
    c2a[i][1]=int(c2a[i][1])
    c2a[i][2]=dt.utcfromtimestamp(int(c2a[i][2])).strftime('%Y-%m-%d')
    
for i in range(0,n):
    c2q[i]=c2q[i].split()
    c2q[i][0]=int(c2q[i][0])
    c2q[i][1]=int(c2q[i][1])
    c2q[i][2]=dt.utcfromtimestamp(int(c2q[i][2])).strftime('%Y-%m-%d')

## Faster version, but gives memory error

In [ ]:
'''
with open ('sx-stackoverflow-a2q.txt') as dataset_a2q:
    a2q = [int(x) for x in dataset_a2q.read().split()] #while reading lines, split and transform into int
    #len(lst)=3* real number of lines, because after this step each value is stored alone: no nested lists
'''

In [ ]:
'''
m=int(len(a2q)/3)
for i in range(2,m+1,3):
    a2q[i]=dt.utcfromtimestamp(a2q[i]).strftime('%Y-%m-%d')
    '''

In [ ]:
'''
with open ('sx-stackoverflow-c2a.txt') as dataset_c2a:
    c2a = [int(x) for x in dataset_c2a.read().split()]
    '''

In [ ]:
'''
m=int(len(c2a)/3)
for i in range(2,m+1,3):
    c2a[i]=dt.utcfromtimestamp(c2a[i]).strftime('%Y-%m-%d')
'''

In [ ]:
'''
with open ('sx-stackoverflow-c2q.txt') as dataset_c2q:
    c2q = [int(x) for x in dataset_c2q.read().split()]
'''

In [ ]:
'''
m=int(len(c2q)/3)
for i in range(2,m+1,3):
    c2q[i]=dt.utcfromtimestamp(c2q[i]).strftime('%Y-%m-%d')
    '''

### Remove answer from user to his own question, i.e. remove loops from corresponding graph

In [ ]:
a2q_n=a2q[0:n]
c2a_n=c2a[0:n]
c2q_n=c2q[0:n]

In [ ]:
w=0
while w<len(a2q_n):
    if a2q_n[w][0]==a2q_n[w][1]:
        del a2q_n[w]
    else:
        w+=1
naq=len(a2q_n)

### Should do the same for the other 2

In [ ]:
x=0
while x<len(c2a_n):
    if c2a_n[x][0]==c2a_n[x][1]:
        del c2a_n[x]
    else:
        x+=1
nca=len(c2a_n)

In [ ]:
len(c2a_n)

In [ ]:
y=0
while y<len(c2q_n):
    if c2q_n[y][0]==c2q_n[y][1]:
        del c2q_n[y]
    else:
        y+=1
ncq=len(c2q_n)

## NOW CREATE WEIGHTED GRAPH ??

### We'll use a nested dictionary

In [ ]:
def dateInRange(date, initTime, endTime): #return true if the given date its in the range [initTime, endTime], return false otherwise
    if(initTime == '-' or endTime == '-'):
        return True
    if initTime <= date <= endTime:
        return True
    else:
        return False
    
def createGraph(initTime, endTime, data):
    graph = {}
    for x in range(len(data)):
        if dateInRange(data[x][2], initTime, endTime):
            if data[x][0] in graph: #we already have the node of that user in the graph
                if data[x][1] in graph[data[x][0]]: #we already have one interaction of user data[x][0] with user data[x][1], so we add 1 to the number of interactions
                    graph[data[x][0]][data[x][1]] += 1
                else:
                    graph[data[x][0]][data[x][1]] = 1 #we don't have an interacion between that 2 users, so we initialize the interaction with a 1 (because till the moment we only have 1 interaction).
            else: #we don't have the node of the user in the graph
                graph[data[x][0]] = {data[x][1] : 1}
    return graph

In [ ]:
'''
auxData = [['0','1','2008-02-11'], ['0', '2', '2008-02-11'], ['0', '1', '2008-02-11'], ['1','0', '2008-04-11']]
dict_a2q_n = createGraph('2008-02-11', '2008-02-11', auxData)
dict_a2q_n
'''

In [ ]:
dict_a2q_n = createGraph('-', '-', a2q_n)
dict_c2a_n = createGraph('-', '-', c2a_n)
dict_c2q_n = createGraph('-', '-', c2q_n)


In [ ]:
'''
dict_a2q_n={}
for i in range (0,naq):
    dict_a2q_n[a2q_n[i][0]]={'a2vs_question':a2q_n[0][1],'time_a2q':a2q_n[0][2],'weight':0}
    '''

In [ ]:
'''
dict_c2a_n={}
for i in range (0,nca):
    dict_c2a_n[c2a_n[i][0]]={'c2vs_answer':c2a_n[0][1],'time_c2a':c2a_n[0][2],'weight':0}
    '''

In [ ]:
'''
dict_c2q_n={}
for i in range (0,ncq):
    dict_c2q_n[c2q_n[i][0]]={'c2vs_question':c2q_n[0][1],'time_c2q':c2q_n[0][2],'weight':0}
    '''

#### For the merged dataset: -----> to work on, according to how we decide to merge

In [ ]:
dict_merged={}
dict_merged.update(dict_a2q_n)
for i in dict_c2a_n.keys():
    if i not in dict_merged.keys():
        dict_merged[i]={'c2vs_answer':c2a_n[0][1],'time_c2a':c2a_n[0][2],'weight':0}
for j in dict_c2q_n.keys():
    if j not in dict_merged.keys(): 
        dict_merged[j]={'c2vs_question':c2q_n[0][1],'time_c2q':c2q_n[0][2],'weight':0}


In [ ]:
for key in dict_merged.keys():
    if len(dict_merged[key])>3:
        print(i)

####Functionalities are defined as methods within a class.

In [ ]:
class Graphs:
    def _init_(self, i):
        # <i> should be used in visualization phase.
        # We call each functionality (method) within the class according to
        # the given <i> by the user.
        self.i = i
        self.results = []

    # First functionality
    def get_features(self, inp_graph):

        # Checking if the graph is undirected or not by looking for
        # similar edges between pairs of vertices.
        def is_directed():
            for u, neigh in inp_graph.items():
                for n in neigh.items():
                    if n[0] in inp_graph:
                        if n[0] != u and not ([(node, w) for node, w in inp_graph[n[0]].items() if (node == u and w == n[1])]):
                            # print("The graph is directed")
                            return 1
            # print("The graph is undirected.")
            return 0

        # Number of users is equal to all the unique values both in keys
        # and values of the given graph in the format of a dictionary.
        def num_of_users():
            _users = []
            for k, v in inp_graph.items():
                _users.append(k)
                for i in [*v]:
                    _users.append(i)
            return len(set(_users))

        # Counting all the values of the sub-dictionaries. If the graph is undirected
        # then we divide the result by two.
        def num_ans():
            _counter = 0
            for i in inp_graph.values():
                _counter += len(i)
            if self.results[0] == 0:
                return _counter/2
            else:
                return _counter

        # Number of edges / number of vertices
        def avg_links():
            return self.results[2]/self.results[1]

        # Two different scenarios based on whether the graph is directed
        # or not
        def density_degree():
            if self.results[0] == 1:
                _density = self.results[2] / (self.results[1]*(self.results[1] - 1))
            else:
                _density = 2 * self.results[2] / (self.results[1] * (self.results[1] - 1))
            return _density

        # if density < 0.5 then it is sparse; otherwise it is considered dense.
        def sparsity():
            if self.results[4] <= 0.5:
                # print("Sparse!")
                return 0
            else:
                # print("Dense!")
                return 1

        self.results.append(is_directed())
        self.results.append(num_of_users())
        self.results.append(num_ans())
        self.results.append(avg_links())
        self.results.append(density_degree())
        self.results.append(sparsity())

        return self.results

    # Second functionality
    def best_users(self, user_id, init_time, end_time, mtrc):
        return pass



# Testing with both directed and undirected (dense) graphs
# uncomment for directed graph
# my_dict = {'A': {'B': 2, 'C': 3}, 'B': {'A': 4, 'C': 5, 'D':10}}
# uncomment for undirected graph
# my_dict = {'A': {'B': 2, 'C': 3}, 'B': {'A': 2, 'C': 5}, 'C':{'A':3, 'B':5}}

# i = 1 indicates functionality 1
# graph_features = Graphs(1)
# report = graph_features.get_features(my_dict)
# print(report)
